In [1]:
# Gerekli Kütüphaneleri Yükleyin
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
data= pd.read_csv(r"C:\Users\hsynt\Downloads\Selcuk Abi\Selcuk Abi\VeriSeti2.csv")

In [3]:
# Özellik ve Hedef Ayrımı
X = data.drop(columns=['Target_10', 'adjclose'])
y = data['Target_10']

# Eğitim ve Test Setlerine Bölme
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Özellikleri Ölçeklendirme
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# PCA ve LDA Dönüşümleri
pca = PCA(n_components=10)
lda = LDA()

X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

X_train_lda = lda.fit_transform(X_train_scaled, y_train)
X_test_lda = lda.transform(X_test_scaled)

# Algoritmalar ve İsimleri
models = {
    "Random Forest": RandomForestClassifier(random_state=42, n_estimators=200),
    "XGBoost": XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss'),
    "SVM": SVC(probability=True, random_state=42),
    "k-NN": KNeighborsClassifier(n_neighbors=5),
    "Logistic Regression": LogisticRegression(random_state=42, max_iter=1000)
}

# Sonuçları Saklamak İçin Listeler
train_results = []
test_results = []

# PCA, LDA ve No Reduction Durumlarını İçeren Modellerin Eğitimi ve Performans Sonuçlarının Hesaplanması
reductions = {
    "No Reduction": (X_train_scaled, X_test_scaled),
    "With PCA": (X_train_pca, X_test_pca),
    "With LDA": (X_train_lda, X_test_lda)
}

for reduction_name, (X_tr, X_te) in reductions.items():
    for model_name, model in models.items():
        # Modeli Eğitme
        model.fit(X_tr, y_train)

        # Eğitim Kümesi Tahminleri
        y_train_pred = model.predict(X_tr)
        y_train_proba = model.predict_proba(X_tr)[:, 1] if hasattr(model, "predict_proba") else y_train_pred

        # Test Kümesi Tahminleri
        y_test_pred = model.predict(X_te)
        y_test_proba = model.predict_proba(X_te)[:, 1] if hasattr(model, "predict_proba") else y_test_pred

        # Eğitim Kümesi Performans Metrikleri
        train_results.append({
            "Reduction": reduction_name,
            "Model": model_name,
            "Accuracy": accuracy_score(y_train, y_train_pred) * 100,
            "Precision": precision_score(y_train, y_train_pred) * 100,
            "Recall": recall_score(y_train, y_train_pred) * 100,
            "F1-Score": f1_score(y_train, y_train_pred) * 100,
            "AUC": roc_auc_score(y_train, y_train_proba) * 100
        })

        # Test Kümesi Performans Metrikleri
        test_results.append({
            "Reduction": reduction_name,
            "Model": model_name,
            "Accuracy": accuracy_score(y_test, y_test_pred) * 100,
            "Precision": precision_score(y_test, y_test_pred) * 100,
            "Recall": recall_score(y_test, y_test_pred) * 100,
            "F1-Score": f1_score(y_test, y_test_pred) * 100,
            "AUC": roc_auc_score(y_test, y_test_proba) * 100
        })

# Sonuçları DataFrame Olarak Gösterme
train_df = pd.DataFrame(train_results)
test_df = pd.DataFrame(test_results)

# Eğitim Kümesi Performans Sonuçlarını Gösterme
print("Eğitim Kümesi Performans Sonuçları (Reduction):")
print(train_df)

# Test Kümesi Performans Sonuçlarını Gösterme
print("\nTest Kümesi Performans Sonuçları (Reduction):")
print(test_df)

C:\Users\hsynt\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [17:45:33] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\hsynt\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [17:59:27] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\hsynt\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [18:08:06] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Eğitim Kümesi Performans Sonuçları (Reduction):
       Reduction                Model    Accuracy   Precision      Recall  \
0   No Reduction        Random Forest  100.000000  100.000000  100.000000   
1   No Reduction              XGBoost   91.341872   91.641713   93.223556   
2   No Reduction                  SVM   82.205934   83.516673   85.467067   
3   No Reduction                 k-NN   86.223545   87.150596   88.778033   
4   No Reduction  Logistic Regression   82.013309   83.283107   85.400848   
5       With PCA        Random Forest  100.000000  100.000000  100.000000   
6       With PCA              XGBoost   88.382449   88.462196   91.422391   
7       With PCA                  SVM   79.844399   81.812634   82.849197   
8       With PCA                 k-NN   84.890179   85.623606   88.133498   
9       With PCA  Logistic Regression   79.461650   81.111590   83.109659   
10      With LDA        Random Forest   99.882424   99.850042   99.942610   
11      With LDA            